In [ ]:
# 使用 python-dotenv 加载 .env 文件
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.messages import HumanMessage, SystemMessage

import os

# 加载当前目录下的 .env 文件
load_dotenv(override=True)

# 获取环境变量
api_key = os.getenv("API_KEY")
base_url = os.getenv("BASE_URL")
model_name = os.getenv("MODEL_NAME")


# 初始化模型
model = ChatOpenAI(
    base_url=base_url,
    model=model_name,
    api_key=api_key,
    temperature=1,
)

In [ ]:
# 基于plan and execute
from pydantic import BaseModel, Field
from typing import List, TypedDict, Annotated, Tuple
import operator
from prompt import PLANNER_SYSTEM_PROMPT, EXECUTOR_SYSTEM_PROMPT
from langchain.agents import create_agent


class AgentState(TypedDict):
    user_query: str  # 用户问题
    steps: List[str]  # 将要执行的计划
    past_steps: Annotated[
        List[Tuple], operator.add
    ]  # 已经执行了的步骤 [step,result]元组
    response: str


class Plan(BaseModel):
    """执行计划"""

    steps: List[str] = Field(description="要遵循的不同步骤，应按顺序排列")


def plan_node(state: AgentState):

    # 构造messages
    messages = [
        SystemMessage(content=PLANNER_SYSTEM_PROMPT),
        HumanMessage(
            f"""
**用户问题：** "{state['steps']}"

**文件信息：**
- **File Path**: `./data.csv`
- **Known Schema**: 无 
(如果 Known Schema 为空，请先制定计划去获取它。)
"""
        ),
    ]

    planer = model.with_structured_output(Plan)

    result = planer.invoke(messages)

    return {steps: result.steps}


def execute_node(state: AgentState):
    # 拿到当前要执行的步骤
    current_plan = state["steps"][0]
    # 拿到之前完成了的步骤和结果
    history_steps = "暂无历史记录(这是第一步)"

    if state["past_steps"]:
        history_steps = "\n".join(
            f"步骤:{step}\n结果:{result}\n" for step, result in state["past_steps"]
        )
    # 构造messages
    messages = [
        SystemMessage(content=EXECUTOR_SYSTEM_PROMPT),
        HumanMessage(
            content=f"""请利用工具执行以下具体任务：

### 1. 项目背景 (Context)
**总体目标**: {state['steps']}

### 2. 参考资料 (Execution History)
*以下是前序步骤的执行结果，请从中提取你需要的参数（如ID、链接、关键数据），不要重复做这些工作：*
----------------------------------------
{history_steps}
----------------------------------------

### 3. 当前任务 (Current Mission)
**请立即执行此步骤**: {current_plan}
"""
        ),
    ]

    executor = create_agent(
        tools=[],
        model=model,
    )

    # 调用执行当前节点
    response = executor.invoke(input={"messages": messages})

    return {"past_steps": [(current_plan, response["messages"][-1].content)]}